## 12. Reading and Preprocessing Images

- 이미지의 기본 형식: `H(Height) × W(Weight) × C(Channel)`
    
    ⇒ 이미지의 channel은 RGB: 3
    
- 처리 과정(CNN 하기 전 조치):
    1. float32로 변환
    2. 텐서화 (Tensor로 변환)
    3. 정규화(Normalization)
    4. 텐서 차원 변경: `C × H * W`
    5. 배치 처리: `B(Batch) * C × H * W`

In [21]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [2]:
import torch
from torchvision import datasets, transforms

# 이미지 변환
transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# CIFAR-10은 10개의 클래스에 걸쳐 총 60,000개의 32x32 컬러 이미지로 구성된 데이터셋
train_dataset = datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms
)
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=128,
    shuffle=True,
)

val_dataset = datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms
)

val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=128,
    shuffle=False,
)

print("Train dataset size:", len(train_dataset))
print("Validation dataset size:", len(val_dataset))



Files already downloaded and verified
Files already downloaded and verified
Train dataset size: 50000
Validation dataset size: 10000


In [19]:
import numpy as np

train_dataset_img, train_dataset_label = train_dataset[3]
train_loader_img, train_loader_label = next(iter(train_loader))

train_dataset_np = np.array(train_dataset_img)
train_loader_np = np.array(train_loader_img)

print(f'train_dataset_img shape: {train_dataset_img.shape}')
print(f'train_loader_img shape: {train_loader_img.shape}')


train_dataset_img shape: torch.Size([3, 32, 32])
train_loader_img shape: torch.Size([128, 3, 32, 32])


In [9]:

import torch.nn as nn

class SimpleCNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super().__init__()
    self.seq_model = nn.Sequential(
      nn.Linear(input_size, hidden_size),
      nn.ReLU(),
      nn.Linear(hidden_size, hidden_size),
      nn.ReLU(),
      nn.Linear(hidden_size, hidden_size),
      nn.ReLU(),
      nn.Dropout(),
      nn.Linear(hidden_size, hidden_size),
      nn.ReLU(),
      nn.Dropout(),
      nn.Linear(hidden_size, output_size)
    )

  def forward(self, x):
    x = x.view(x.size(0), -1)
    return self.seq_model(x)

In [ ]:
mynet = SimpleCNN(input_size=32*32*3, hidden_size=128, output_size=10)
mynet = mynet.to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mynet.parameters(), lr=0.001)

for epoch in range(100):
  mynet.train()
  for batch_idx, (data, label) in enumerate(train_loader):
    data, label = data.to(device), label.to(device)
    optimizer.zero_grad()
    output = mynet(data)
    train_loss = loss_fn(output, label)
    train_loss.backward()
    optimizer.step()

  mynet.eval()
  val_loss = 0
  correct = 0
  with torch.no_grad():
    for data, label in val_loader:
      data, label = data.to(device), label.to(device)
      output = mynet(data)
      val_loss += loss_fn(output, label).item()
      pred = output.argmax(dim=1, keepdim=True)
      correct += pred.eq(label.view_as(pred)).sum().item()
  
  val_loss /= len(val_loader.dataset)
  accuracy = 100. * correct / len(val_loader.dataset)

  print(f'Epoch {epoch+1}, Loss: {train_loss.item():.4f}, Val Loss: {val_loss:.4f}, Accuracy: {accuracy:.2f}%')
    


Epoch 1, Loss: 1.5829, Val Loss: 0.0127, Accuracy: 43.49%
Epoch 2, Loss: 1.6539, Val Loss: 0.0119, Accuracy: 46.78%
Epoch 3, Loss: 1.4673, Val Loss: 0.0114, Accuracy: 49.13%
Epoch 4, Loss: 1.3113, Val Loss: 0.0111, Accuracy: 50.79%
Epoch 5, Loss: 1.4199, Val Loss: 0.0110, Accuracy: 51.55%
Epoch 6, Loss: 1.4756, Val Loss: 0.0109, Accuracy: 51.39%
Epoch 7, Loss: 1.1572, Val Loss: 0.0108, Accuracy: 52.06%
Epoch 8, Loss: 1.3610, Val Loss: 0.0108, Accuracy: 51.65%
Epoch 9, Loss: 1.2513, Val Loss: 0.0108, Accuracy: 51.96%
Epoch 10, Loss: 1.0685, Val Loss: 0.0109, Accuracy: 52.68%
Epoch 11, Loss: 1.1445, Val Loss: 0.0108, Accuracy: 53.07%
Epoch 12, Loss: 1.2400, Val Loss: 0.0108, Accuracy: 52.73%
Epoch 13, Loss: 0.9693, Val Loss: 0.0111, Accuracy: 52.31%
Epoch 14, Loss: 1.2860, Val Loss: 0.0109, Accuracy: 53.00%
Epoch 15, Loss: 1.1764, Val Loss: 0.0112, Accuracy: 52.39%
Epoch 16, Loss: 0.8532, Val Loss: 0.0112, Accuracy: 52.75%
Epoch 17, Loss: 0.8936, Val Loss: 0.0112, Accuracy: 53.17%
Epoch 